# AB - Module 2

In [3]:
from scipy.stats import binom
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Задача 5
Теперь представим, что мы обзвонили 1000 клиентов и хотим проверить, с какой вероятностью 100 и меньше пользователей согласятся продлить нашу подписку.

$$p=0.3$$

In [26]:
x = np.linspace(0, 1, 100)
binom.cdf(100, 1000, 0.3)

1.7183404003317793e-52

## Задача 13

Предположим, что у нас есть 25 магазинов. Средняя выручка в день составляет 170 тысяч рублей, а оценённое по этой выборке среднеквадратичное отклонение составляет 12 тысяч рублей. Оцените 95%-й доверительный интервал средней выручки магазинов.

Для выполнения задания вам могут быть полезны функции для распределений из модуля `scipy.stats`.


In [32]:
from scipy.stats import t

In [34]:
mu = 170_000
sigma = 12_000
n = 25
alpha = 0.05

delta = t.ppf(1 - alpha / 2, df=n-1) * sigma / np.sqrt(n)

mu - delta, mu + delta

(165046.64345209274, 174953.35654790726)

## Задача 14

Давайте ещё попрактикуемся в построении доверительного интервала. В качестве выборки будем брать синтетические данные — сгенерируйте 1000 чисел из распределения Пуассона с параметром 50. Вам будет полезен модуль random в библиотеке numpy.

Посчитайте 95-й доверительный интервал среднего на основе центральной предельной теоремы. Какая ширина интервала у вас получилась? Среди вариантов ответа выбирайте ближайший к вашему.

In [35]:
data = np.random.poisson(lam=60, size=1000)

In [38]:
mu = 60
sigma = np.std(data, ddof=1)

In [39]:
n = 1000

In [40]:
delta = 1.96 * sigma / np.sqrt(n)

In [41]:
mu - delta, mu + delta

(59.51482597725564, 60.48517402274436)

## Задание 15

Воспользуемся тем же набором данных (синтетическим из распределения Пуассона).

Постройте 95-й доверительный интервал на среднеквадратичное отклонение с помощью бутстрэпа. Рекомендуется попробовать сделать это вручную (генерировать бутстрапированные выборки в цикле, сохранять нужную статистику, вычислять квантили вычисленного ряда статистик) и с помощью соответствующей функции из модуля stats библиотеки scipy. Внимание, у функции bootstrap есть параметр method, который определяет, как будут вычисляться границы доверительного интервала. В нашем случае это был percentile.

На какой интервал больше всего похож ваш?

In [43]:
from scipy.stats import bootstrap

bootstrap(
    data=(data,),
    statistic=np.mean,
    confidence_level=0.95,
    n_resamples=1000,
    method='percentile',
).confidence_interval

ImportError: cannot import name 'bootstrap'

'0.19.0'

In [46]:
from sklearn.datasets import fetch_california_housing

X, y = fetch_california_housing(return_X_y=True, as_frame=True)

TypeError: fetch_california_housing() got an unexpected keyword argument 'return_X_y'